# Data Scraping
**CRIMES VIOLENTOS LETAIS E INTENCIONAIS – CVLI (SSPDS/CE)**  
>Autor:       Erivando Sena  
E-mail:      erivandosena@gmail.com   
Criado:        03/08/2019  
Atualizado:  14/02/2023  

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/erivandosena/web-scraping-r/blob/master/DataScraping-SSPDS-CE.ipynb)

### Setup R and packages

In [2]:
suppressPackageStartupMessages({
  if (!requireNamespace("reticulate", quietly = TRUE)) install.packages("reticulate")
  if (reticulate::py_module_available("google.colab")) {
    cat("Environment Google Colab.\n")

    path_rlibrary <- file.path(.libPaths()[2])
    .libPaths(c("/root/miniconda3/lib/R/library", .libPaths()))
  } else {
    cat("Other Environment.\n")

    # Add R in path of system
    path_r <- normalizePath(file.path(R.home("bin")))
    current_path <- Sys.getenv("PATH")
    if (!grepl(paste0(";", path_r, ";"), paste0(";", current_path, ";"))) {
        Sys.setenv(PATH = paste(path_r, current_path, sep = ";"))
    }

    path_ruser <- file.path(Sys.getenv("LOCALAPPDATA"), "R", fsep="\\")
    Sys.setenv(R_USER=path_ruser)

    path_rlibrary <- file.path(Sys.getenv("LOCALAPPDATA"), "R", "win-library", fsep="\\")
    dir.create(path_rlibrary, recursive = TRUE, showWarnings = FALSE)
    .libPaths(c(normalizePath(path_rlibrary), .libPaths()))

    # Save history
    .First <- function() {
      history_file <- file.path(Sys.getenv("R_USER"), ".Rhistory")
      if (file.exists(history_file)) {
        source(history_file)
      }
    }
    .Last <- function() {
      history_file <- file.path(Sys.getenv("R_USER"), ".Rhistory")
      savehistory(history_file)
    }

    # Setup JAVA_HOME
    Sys.setenv(JAVA_HOME = normalizePath(file.path(Sys.getenv("SystemDrive"), "Program Files", "Java", "jdk1.8.0_202")))
    java_path <- normalizePath(file.path(Sys.getenv("JAVA_HOME"), "bin"))
    Sys.setenv(PATH = paste(java_path, Sys.getenv("PATH"), sep = ";"))

    cat(paste(path_ruser, "\n"))
  }

  # Update packages R
  if (length(old.packages()) > 0) {
      update.packages(lib.loc = path_rlibrary, repos='http://cran.rstudio.com/', ask=FALSE, checkBuilt=FALSE)
  }
})

# Check information
cat(paste(system2("java", args = c("-version"), stdout = TRUE, stderr = TRUE), "\n"))
.libPaths()
Sys.which("R")

Other Environment.
C:\Users\Erivando\AppData\Local\R 
java version "1.8.0_202" 
 Java(TM) SE Runtime Environment (build 1.8.0_202-b08) 
 Java HotSpot(TM) 64-Bit Server VM (build 25.202-b08, mixed mode) 


[1] "C:/Users/Erivando/AppData/Local/R/win-library"    
[2] "C:/Users/Erivando/AppData/Local/R/win-library/4.3"
[3] "C:/Program Files/R/R-4.3.0/library"

R 
"C:\\PROGRA~1\\R\\R-43~1.0\\bin\\x64\\R.exe"

In [ ]:
# sessionInfo()

In [3]:
pacotes_comuns <- c("remotes","installr","tidyverse","rvest","stringi","readxl","lubridate","xml2", "leaflet",
                    "htmlwidgets","ggthemes","forecast","prophet", "mice","magrittr","kableExtra","ggmap","stringr",
                    "purrr","dplyr","readr")
pacotes_util <- c("rJava")
pacotes_extra <- c("pdftools","ropensci/tabulizer")
pacotes <- c(pacotes_comuns, pacotes_util, pacotes_extra)

suppressPackageStartupMessages({
  if (reticulate::py_module_available("google.colab")) {
    cat("Environment Google Colab.\n")

    # Environment Conda
    system("apt-get update && apt-get install -y libpoppler-cpp-dev r-base openjdk-11-jdk")
    Sys.setenv(LD_LIBRARY_PATH = "/usr/lib/jvm/java-11-openjdk-amd64/lib")
    if (!file.exists("~/miniconda3")) {
      system("mkdir -p ~/miniconda3")
      system("wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh")
      system("bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3")
      system("sudo ~/miniconda3/bin/conda init bash")
      system("sudo ~/miniconda3/bin/conda config --add channels conda-forge")
      system("sudo ~/miniconda3/bin/conda config --set channel_priority strict")
    }
    cat(system("sudo ~/miniconda3/bin/conda install --yes r-rjava", intern = TRUE), sep = "\n")
    #cat(system("sudo ~/miniconda3/bin/conda install --yes r-pdftools", intern = TRUE), sep = "\n")
    #cat(system("sudo ~/miniconda3/bin/conda install --yes r-tabulizer", intern = TRUE), sep = "\n")

    for (pkg in pacotes_extra) {
      pkg_extra <- sub("ropensci/", "", pkg)
      if (!(pkg_extra %in% rownames(installed.packages()))){
        if (grepl("/", pkg)) {
          remotes::install_github(pkg, INSTALL_opts = "--no-multiarch")
        } else {
          install.packages(pkg)
        }
      }
    }
    for (pkg in pacotes_extra) {
      pkg_extra <- sub("ropensci/", "", pkg)
      if ((pkg_extra %in% rownames(installed.packages()))){
        library(pkg_extra, character.only = TRUE)
      }
    }

  } else {

    cat("Other Environment.\n")
    for (pkg in pacotes) {
      pkg_extra <- sub("ropensci/", "", pkg)
      if (!(pkg_extra %in% rownames(installed.packages()))){
        if (grepl("/", pkg)) {
          remotes::install_github(pkg, INSTALL_opts = "--no-multiarch")
        } else {
          install.packages(pkg)
        }
      }
    }
    for (pkg in pacotes) {
      pkg_extra <- sub("ropensci/", "", pkg)
      if ((pkg_extra %in% rownames(installed.packages()))){
        library(pkg_extra, character.only = TRUE)
      }
    }
  }
})

Other Environment.


Warning message:
"package 'remotes' was built under R version 4.3.2"
Warning message:
"package 'installr' was built under R version 4.3.2"
Warning message:
"package 'tidyverse' was built under R version 4.3.2"
Warning message:
"package 'ggplot2' was built under R version 4.3.2"
Warning message:
"package 'tibble' was built under R version 4.3.2"
Warning message:
"package 'tidyr' was built under R version 4.3.2"
Warning message:
"package 'readr' was built under R version 4.3.2"
Warning message:
"package 'purrr' was built under R version 4.3.2"
Warning message:
"package 'dplyr' was built under R version 4.3.2"
Warning message:
"package 'stringr' was built under R version 4.3.2"
Warning message:
"package 'forcats' was built under R version 4.3.2"
Warning message:
"package 'lubridate' was built under R version 4.3.2"
Warning message:
"package 'rvest' was built under R version 4.3.2"
Warning message:
"package 'stringi' was built under R version 4.3.2"
Warning message:
"package 'readxl' was b

### Folders configuration

In [4]:
dir_docs <- c("Documentos")
dir_dados <- c("Dados")
# Vetor contendo o camainho absoluto dos diretorios
diretorios <- c(dir_dados, dir_docs)
for (diretorio in diretorios) {
  if(!dir.exists(diretorio)) {
    dir.create(diretorio, recursive = TRUE)
  }
}

### Functions

In [5]:
# Extrai informacoes de ano do código html
extrai_lista_anos <- function(url_site) {
  pagina_html <- xml2::read_html(url_site)

  df_lista_urls <- pagina_html %>%
    rvest::html_nodes('.grid a.box') %>%
    purrr::map(xml2::xml_attrs) %>%
    purrr::map_df(~as.list(.)) %>%
    na.omit(.) %>%
    .[-c(nrow(.)),]

  df_lista_titulos <- pagina_html %>%
    rvest::html_nodes('.grid p') %>%
    rvest::html_text() %>%

    gsub(" ", "", .) %>%
    substr(., nchar(.)-3, nchar(.)) %>%
    .[-length(.)] %>% # remove ano 2013
    as.data.frame(.) %>%
    setNames(., "ano")

  # Juntar os dataframes df_lista_urls e df_lista_titulos
  df_urls_anos <- data.frame(df_lista_urls, df_lista_titulos)
  excluir <- c("class", "target")
  df_urls_anos <- df_urls_anos[,!(names(df_urls_anos) %in% excluir)]
  df_urls_anos[,1][df_urls_anos[, 1] == "#"] <- NA

  return(df_urls_anos)
}

# Baixa lista de arquivos por ano
obtem_arquivos <- function(url_site, anos) {
  pastas <- list.dirs(dir_docs, full.names = TRUE, recursive = FALSE)
  unlink(pastas, recursive = TRUE)

  for(indice in 1:nrow(anos)) {
    print(paste("Lendo codigo HTML da pagina Web:", anos[indice,1], "-", anos[indice,2], sep = " "))
    df_listas <- extrai_lista_documentos(anos[indice,1])
    Sys.sleep(1)
    download_documentos(df_listas)
    Sys.sleep(10)
  }
}

# Extrai informacoes de docs do codigo html
extrai_lista_documentos <- function(url_pagina) {
  pagina_html <- xml2::read_html(url_pagina)

  df_lista_urls <- pagina_html %>%
    rvest::html_nodes('.-Verde a.box') %>%
    purrr::map(xml2::xml_attrs) %>%
    purrr::map_df(~as.list(.))

  df_lista_titulos <- pagina_html %>%
    rvest::html_nodes('.-Verde h3') %>%
    rvest::html_text() %>%
    remove_acentos(.) %>%
    gsub("–", " - ", .) %>%
    gsub("/", " - ", .) %>%
    gsub(" ", "", .) %>%
    as.data.frame(.) %>%
    setNames(., "mes")

  # Juntar os dataframes df_lista_urls e df_lista_titulos
  df_urls_nomes <- data.frame(df_lista_urls, df_lista_titulos)
  excluir <- c("class", "target", "rel")
  df_urls_nomes <- df_urls_nomes[,!(names(df_urls_nomes) %in% excluir)]
  df_urls_nomes[,1][df_urls_nomes[, 1] == "#"] <- NA

  return(df_urls_nomes)
}

# Baixa os arquivos da web para uma pasta local
download_documentos <- function(d_frame_lista) {
  anos <- extrai_lista_anos(URL_site)
  for (titulo in d_frame_lista$mes) {
    df_url <- select(filter(d_frame_lista, d_frame_lista$mes == titulo), "href")
    df_nome <- select(filter(d_frame_lista, d_frame_lista$mes == titulo), "mes")
    for(ano in anos$ano) {
      if(ano == substr(df_nome$mes, nchar(as.character(df_nome$mes))-3, nchar(as.character(df_nome$mes)))) {
        diretorio <- file.path(".", dir_docs, ano)
        if(!dir.exists(diretorio)) {
          dir.create(diretorio, recursive = TRUE)
        }
        nome_arquivo_completo <- obtem_nome_arquivos(ano, df_nome$mes)
        if(file.exists(nome_arquivo_completo)) {
          print(paste0("Arquivo baixado: ", df_nome$mes))
        } else {
          if (!is.na(df_url$href)) {
            download.file(df_url$href, destfile = nome_arquivo_completo, mode = "wb", quiet=FALSE)
          } else {
            print(paste("Arquivo indisponivel: ", df_nome$mes, sep = " "))
          }
        }
      }
    }
  }
}

# Formata nome dos arquivos
obtem_nome_arquivos <- function(dir_superior, nome_relacao) {
  nome_arquivo <- nome_relacao %>% gsub("[/. ,]","_", .)
  path <- file.path(".", dir_docs, dir_superior, paste0(nome_arquivo, ".pdf"))
  return(path)
}

# Remover acentos
remove_acentos <- function(obj_str) {
  if(!is.character(obj_str)) {
    obj_str <- as.character(obj_str)
  }
  obj_str <- stringi::stri_trans_general(str = obj_str, "latin-ascii")
  return(obj_str)
}

# Aplicar transformações em data frame
processar_dataframe <- function(df) {
  df <- df %>% as.data.frame()
  df <- df[,-1]
  if (ncol(df) >= 7) {
    df <- df %>%
      select(1:7) %>%
      setNames(cabecalho)
      df <- df %>% mutate_all(~ifelse(. == "", NA, .))
      df <- na.omit(df)
      df <- df %>% mutate(across(everything(), as.character))
    return(df)
  }
}

### Collect of data

In [6]:
URL_site <- "https://www.sspds.ce.gov.br/estatisticas-2-2-2-2-2-2/"
anos <- extrai_lista_anos(URL_site)

anos_filtro <- anos[anos$ano %in% c("2023","2024"), ] # Year 2023 only, keep c("2023")
# anos_filtro <- anos # Enabled all years

obtem_arquivos(URL_site, anos_filtro)
vetor_anos <- as.numeric(anos_filtro$ano)
lista_df <- list()

for (iano in seq_along(vetor_anos)) {
  data_frame_meses <- extrai_lista_documentos(anos_filtro[iano, 1])
  num_docs <- nrow(data_frame_meses)
  for (ndoc in 1:num_docs) {
    arquivo <- file.path(dir_docs, vetor_anos[iano], paste0(data_frame_meses[ndoc, 2], ".pdf"))
    paginas <- pdf_info(arquivo)$pages
    for (npage in 1:paginas) {
      tabela <- extract_tables(file = arquivo, pages = npage, method = "stream", guess = TRUE, output = "data.frame", encoding = "UTF-8")
      lista_df[[length(lista_df) + 1]] <- tabela
    }
  }
}

[1] "Lendo codigo HTML da pagina Web: https://www.sspds.ce.gov.br/estatisticas-2023 - 2023"


### Processing of data

In [7]:
# Clear data and merge data frame
arquivos_csv <- list.files(dir_dados, pattern = "\\.csv$", full.names = TRUE)
file.remove(arquivos_csv)

df_final <- data.frame()
cabecalho = c("AIS","MUNICIPIO_HOMICIDIO", "NATUREZA_HOMICIDIO", "ARMA_UTILIZADA", "DATA_HOMICIDIO", "SEXO", "IDADE")
lista_df_processada <- lapply(lista_df, processar_dataframe)

for (i in seq_along(lista_df_processada)) {
  df <- lista_df_processada[[i]]
  df_final <- bind_rows(df_final, df)
  # write.csv(df, file = paste0(file.path(dir_dados, paste0(i,"cvli.csv"))), row.names = FALSE) # Enable export the csv of data frames unitary
}

df_final <- df_final %>% arrange(as.Date(DATA_HOMICIDIO, format="%d/%m/%Y")) %>% mutate(ID = c(1:nrow(.))) #%>% .[,c(8,1:ncol(.))]
df_final <- df_final %>% .[,c(8,1:ncol(.)-1)]

# Export dataset
write.csv(df_final, file = paste0(file.path(dir_dados, "cvli.csv")), row.names = FALSE)

[1] TRUE

ID,AIS,MUNICIPIO_HOMICIDIO,NATUREZA_HOMICIDIO,ARMA_UTILIZADA,DATA_HOMICIDIO,SEXO,IDADE
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,AIS 23,Paracuru,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Feminino,24
2,AIS 23,Paracuru,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Feminino,16
3,AIS 07,Fortaleza,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Masculino,20
4,AIS 19,Missão Velha,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Masculino,42
5,AIS 19,Crato,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Masculino,26
6,AIS 18,Jaguaribe,FEMINICÍDIO,Arma branca,01/01/2023,Feminino,27
7,AIS 25,Pacajus,HOMICIDIO DOLOSO,Arma branca,01/01/2023,Masculino,53
8,AIS 13,Aquiraz,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Masculino,41
9,AIS 11,Caucaia,HOMICIDIO DOLOSO,Arma de fogo,01/01/2023,Masculino,31


### Analysis of dataset

In [8]:
# Import dataset
dados_cvli <- read_csv(file = paste0(file.path(dir_dados, "cvli.csv")), show_col_types = FALSE)
print(dados_cvli)

# A tibble: 2,845 × 8
      ID AIS    MUNICIPIO_HOMICIDIO NATUREZA_HOMICIDIO ARMA_UTILIZADA
   <dbl> <chr>  <chr>               <chr>              <chr>         
 1     1 AIS 23 Paracuru            HOMICIDIO DOLOSO   Arma de fogo  
 2     2 AIS 23 Paracuru            HOMICIDIO DOLOSO   Arma de fogo  
 3     3 AIS 07 Fortaleza           HOMICIDIO DOLOSO   Arma de fogo  
 4     4 AIS 19 Missão Velha        HOMICIDIO DOLOSO   Arma de fogo  
 5     5 AIS 19 Crato               HOMICIDIO DOLOSO   Arma de fogo  
 6     6 AIS 18 Jaguaribe           FEMINICÍDIO        Arma branca   
 7     7 AIS 25 Pacajus             HOMICIDIO DOLOSO   Arma branca   
 8     8 AIS 13 Aquiraz             HOMICIDIO DOLOSO   Arma de fogo  
 9     9 AIS 11 Caucaia             HOMICIDIO DOLOSO   Arma de fogo  
10    10 AIS 06 Fortaleza           HOMICIDIO DOLOSO   Arma de fogo  
# ℹ 2,835 more rows
# ℹ 3 more variables: DATA_HOMICIDIO <chr>, SEXO <chr>, IDADE <chr>
